<a href="https://colab.research.google.com/github/Siriyakon-Saenpor/229351_Statistical-Learning-for-Data-Science-1/blob/main/%E0%B8%AA%E0%B8%B3%E0%B9%80%E0%B8%99%E0%B8%B2%E0%B8%82%E0%B8%AD%E0%B8%87_Lab02_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve

# For Fashion-MNIST
from tensorflow.keras.datasets import fashion_mnist

# For 20 Newsgroups
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## Part 1: Marketing Campaign Dataset - Manual Data Preprocessing & Logistic Regression

### Load the Marketing Campaign Dataset ([Data Information](https://archive.ics.uci.edu/dataset/222/bank+marketing))

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be (`'yes'`) or not (`'no'`) subscribed.

In [2]:
bank_url = 'https://raw.githubusercontent.com/donlap/ds352-labs/main/bank.csv'

df = pd.read_csv(bank_url, sep=';', na_values=['unknown'])
df = df.drop(["emp.var.rate", "cons.price.idx", "cons.conf.idx",	"euribor3m", "nr.employed"], axis=1)
print("Shape of the dataset:", df.shape)
df.head()

Shape of the dataset: (41188, 16)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,no


### Data Exploration

In [4]:
print("--- Unique Values for Categorical Columns ---")
for col in df.select_dtypes(include='object').columns:
    print(f"\n'{col}' unique values:")
    print(df[col].value_counts(dropna=False)) # Include NaN counts

--- Unique Values for Categorical Columns ---

'job' unique values:
job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
NaN                330
Name: count, dtype: int64

'marital' unique values:
marital
married     24928
single      11568
divorced     4612
NaN            80
Name: count, dtype: int64

'education' unique values:
education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
NaN                     1731
illiterate                18
Name: count, dtype: int64

'default' unique values:
default
no     32588
NaN     8597
yes        3
Name: count, dtype: int64

'housing' unique values:
housing
yes    21576
no     18622
NaN      990
Name: count, dtype: int64


In [3]:
print("--- Missing Values Count ---")
print(df.isnull().sum())

--- Missing Values Count ---
age               0
job             330
marital          80
education      1731
default        8597
housing         990
loan            990
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
y                 0
dtype: int64


### Data Preprocessing

In [5]:
# Map target variable 'y' to 0 (no) and 1 (yes)
df['y_new'] = df['y'].map({'yes': 1, 'no': 0}) # Write your code here => สร้างคอลัมใหม่


# Drop 'duration' due to data leakage


# Define features (X) and target (y)
y = df['y_new']
x = df.drop(['y', 'y_new'], axis=1)

print(y)
print(x)



0        0
1        0
2        0
3        0
4        0
        ..
41183    1
41184    0
41185    0
41186    1
41187    0
Name: y_new, Length: 41188, dtype: int64
       age          job  marital            education default housing loan    contact month day_of_week  duration  campaign  pdays  previous     poutcome
0       56    housemaid  married             basic.4y      no      no   no  telephone   may         mon       261         1    999         0  nonexistent
1       57     services  married          high.school     NaN      no   no  telephone   may         mon       149         1    999         0  nonexistent
2       37     services  married          high.school      no     yes   no  telephone   may         mon       226         1    999         0  nonexistent
3       40       admin.  married             basic.6y      no      no   no  telephone   may         mon       151         1    999         0  nonexistent
4       56     services  married          high.school      no      n

In [6]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
X_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
24376,39,blue-collar,divorced,high.school,no,no,yes,cellular,nov,mon,190,2,999,0,nonexistent
17119,50,admin.,married,university.degree,no,yes,no,cellular,jul,fri,127,1,999,0,nonexistent
17195,54,technician,married,professional.course,no,no,no,cellular,jul,fri,796,1,999,0,nonexistent
5795,39,admin.,single,high.school,NaN,no,no,telephone,may,mon,399,2,999,0,nonexistent
18430,49,technician,married,basic.9y,no,yes,no,cellular,jul,thu,183,1,999,0,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4460,42,blue-collar,married,basic.4y,NaN,yes,no,telephone,may,tue,78,2,999,0,nonexistent
17132,56,self-employed,married,basic.9y,NaN,no,no,cellular,jul,fri,571,1,999,0,nonexistent
24156,41,blue-collar,married,basic.9y,NaN,yes,no,cellular,nov,mon,176,1,999,0,nonexistent
38469,29,admin.,single,university.degree,no,no,no,cellular,oct,mon,87,2,999,1,failure


We will apply `StandardScaler()`, `OrdinalEncoder()`, and `OneHotEncoder()` on a few selected columns.

**1. Numerical Feature: `age` and `campaign` (Standard Scaling)**

In [7]:
num_cols_demo = ['age', 'campaign']

scaler = StandardScaler()

# Fit the scaler ONLY on the training data

X_train[num_cols_demo] = scaler.fit_transform(X_train[num_cols_demo])
X_test[num_cols_demo] = scaler.transform(X_test[num_cols_demo])

X_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
24376,-0.097850,blue-collar,divorced,high.school,no,no,yes,cellular,nov,mon,190,-0.203242,999,0,nonexistent
17119,0.957809,admin.,married,university.degree,no,yes,no,cellular,jul,fri,127,-0.564213,999,0,nonexistent
17195,1.341685,technician,married,professional.course,no,no,no,cellular,jul,fri,796,-0.564213,999,0,nonexistent
5795,-0.097850,admin.,single,high.school,NaN,no,no,telephone,may,mon,399,-0.203242,999,0,nonexistent
18430,0.861840,technician,married,basic.9y,no,yes,no,cellular,jul,thu,183,-0.564213,999,0,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4460,0.190057,blue-collar,married,basic.4y,NaN,yes,no,telephone,may,tue,78,-0.203242,999,0,nonexistent
17132,1.533623,self-employed,married,basic.9y,NaN,no,no,cellular,jul,fri,571,-0.564213,999,0,nonexistent
24156,0.094088,blue-collar,married,basic.9y,NaN,yes,no,cellular,nov,mon,176,-0.564213,999,0,nonexistent
38469,-1.057540,admin.,single,university.degree,no,no,no,cellular,oct,mon,87,-0.203242,999,1,failure


In [8]:
X_train.describe()

,age,duration,campaign,pdays,previous
count,2.883100e+04,28831.000000,2.883100e+04,28831.000000,28831.000000
mean,-1.813879e-16,257.919462,3.943215e-17,961.665846,0.175436
std,1.000017e+00,259.119798,1.000017e+00,188.890680,0.501287
min,-2.209167e+00,0.000000,-5.642133e-01,0.000000,0.000000
25%,-7.696326e-01,103.000000,-5.642133e-01,999.000000,0.000000
50%,-1.938187e-01,179.000000,-2.032415e-01,999.000000,0.000000
75%,6.699022e-01,319.000000,1.577303e-01,999.000000,0.000000
max,5.564321e+00,4918.000000,1.928924e+01,999.000000,7.000000


Let's take a look at the transformed `age` and `campaign` features and their statistics.

In [ ]:
print("\nOriginal X_train 'age' and 'campaign' head:")
print(X_train[num_cols_demo].head())
print("\nScaled X_train 'age' and 'campaign' head:")
print(pd.DataFrame(X_train_scaled_demo, columns=num_cols_demo, index=X_train.index).head())

print("\nMean of scaled 'age' (train):", X_train_scaled_demo[:, 0].mean())
print("Std Dev of scaled 'campaign' (train):", X_train_scaled_demo[:, 1].std())


Original X_train 'age' and 'campaign' head:
            age  campaign
32218 -1.729190 -0.563259
17124 -0.099251 -0.563259
21805  0.859537 -0.563259
24660 -0.482766 -0.206562
17026  0.859537  5.857275

Scaled X_train 'age' and 'campaign' head:


NameError: name 'X_train_scaled_demo' is not defined

**2. Ordinal Feature: `education` (Ordinal Encoding with Imputation)**

- **Imputation**

In [33]:
ord_col_demo = ['education']

imputer_ord = SimpleImputer(strategy='most_frequent')

X_train[ord_col_demo] = imputer_ord.fit_transform(X_train[ord_col_demo])
X_test[ord_col_demo] = imputer_ord.transform(X_test[ord_col_demo])

X_train['education']

,education
0,4.0
1,5.0
2,6.0
3,4.0
4,6.0
...,...
28826,4.0
28827,2.0
28828,1.0
28829,3.0


- **Ordinal Encoding**

In [10]:
education_categories = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

In [11]:
ordinal_encoder = OrdinalEncoder(categories=[education_categories])

## Write your code here

X_train[ord_col_demo] = ordinal_encoder.fit_transform(X_train[ord_col_demo])
X_test[ord_col_demo] = ordinal_encoder.transform(X_test[ord_col_demo])

X_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
24376,-0.097850,blue-collar,divorced,4.0,no,no,yes,cellular,nov,mon,190,-0.203242,999,0,nonexistent
17119,0.957809,admin.,married,6.0,no,yes,no,cellular,jul,fri,127,-0.564213,999,0,nonexistent
17195,1.341685,technician,married,5.0,no,no,no,cellular,jul,fri,796,-0.564213,999,0,nonexistent
5795,-0.097850,admin.,single,4.0,NaN,no,no,telephone,may,mon,399,-0.203242,999,0,nonexistent
18430,0.861840,technician,married,3.0,no,yes,no,cellular,jul,thu,183,-0.564213,999,0,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4460,0.190057,blue-collar,married,1.0,NaN,yes,no,telephone,may,tue,78,-0.203242,999,0,nonexistent
17132,1.533623,self-employed,married,3.0,NaN,no,no,cellular,jul,fri,571,-0.564213,999,0,nonexistent
24156,0.094088,blue-collar,married,3.0,NaN,yes,no,cellular,nov,mon,176,-0.564213,999,0,nonexistent
38469,-1.057540,admin.,single,6.0,no,no,no,cellular,oct,mon,87,-0.203242,999,1,failure


Let's take a look at the imputed and ordinal-encoded `education`.

In [ ]:
print("\nOriginal X_train 'education' head:")
print(X_train[ord_col_demo].iloc[20:25])
print("\nImputed X_train 'education' head (after imputer.transform):")
print(pd.DataFrame(X_train_imputed_ord_demo, columns=ord_col_demo, index=X_train.index).iloc[20:25])
print("\nOrdinal Encoded X_train 'education' head:")
print(pd.DataFrame(X_train_ord_encoded_demo, columns=ord_col_demo, index=X_train.index).iloc[20:25])

**3. Nominal Feature: `job` (One-Hot Encoding with Imputation)**

- **Imputation**

In [12]:
nom_col_demo = ['job']

imputer_nom = SimpleImputer(strategy='most_frequent')
imputer_nom.fit(X_train[nom_col_demo])

X_train[nom_col_demo] = imputer_nom.fit_transform(X_train[nom_col_demo])
X_test[nom_col_demo] = imputer_nom.transform(X_test[nom_col_demo])

- **Nominal Encoding**

In [13]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

## Write your code here


X_train_onehot = onehot_encoder.fit_transform(X_train[['job']])
X_test_onehot = onehot_encoder.transform(X_test[['job']])

X_test_onehot.shape

(12357, 11)

In [14]:
X_train_onehot

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [15]:
X_train = pd.concat([X_train.reset_index(drop=True),
                     pd.DataFrame(X_test_onehot, columns=onehot_encoder.get_feature_names_out(['job']))], axis=1)

X_train

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed
0,1.437654,retired,married,4.0,no,yes,yes,cellular,jun,fri,186,-0.203242,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.769633,services,married,5.0,no,yes,no,telephone,may,tue,215,0.157730,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.673664,entrepreneur,married,6.0,no,yes,yes,cellular,jul,mon,285,-0.203242,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.533623,admin.,married,4.0,no,no,no,telephone,jun,fri,489,-0.203242,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.453003,retired,married,6.0,no,no,no,cellular,dec,thu,103,-0.564213,999,1,failure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28826,1.149747,unemployed,married,4.0,NaN,yes,no,telephone,jun,tue,70,1.240646,999,0,nonexistent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28827,-0.481726,blue-collar,married,2.0,no,no,no,telephone,may,thu,102,-0.564213,999,0,nonexistent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28828,1.341685,blue-collar,married,1.0,no,no,no,cellular,apr,thu,104,-0.564213,999,1,failure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28829,0.573933,blue-collar,married,3.0,no,no,yes,telephone,may,tue,394,-0.564213,999,0,nonexistent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
X_test = pd.concat([X_test.reset_index(drop=True),
                     pd.DataFrame(X_test_onehot, columns=onehot_encoder.get_feature_names_out(['job']))], axis=1)

X_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed
0,-0.097850,blue-collar,divorced,4.0,no,no,yes,cellular,nov,mon,190,-0.203242,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.957809,admin.,married,6.0,no,yes,no,cellular,jul,fri,127,-0.564213,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.341685,technician,married,5.0,no,no,no,cellular,jul,fri,796,-0.564213,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.097850,admin.,single,4.0,NaN,no,no,telephone,may,mon,399,-0.203242,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.861840,technician,married,3.0,no,yes,no,cellular,jul,thu,183,-0.564213,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12352,0.190057,blue-collar,married,1.0,NaN,yes,no,telephone,may,tue,78,-0.203242,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12353,1.533623,self-employed,married,3.0,NaN,no,no,cellular,jul,fri,571,-0.564213,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
12354,0.094088,blue-collar,married,3.0,NaN,yes,no,cellular,nov,mon,176,-0.564213,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12355,-1.057540,admin.,single,6.0,no,no,no,cellular,oct,mon,87,-0.203242,999,1,failure,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print("\nOriginal X_train 'job' head:")
print(X_train[nom_col_demo].iloc[40:45])
print("\nImputed X_train 'job' head (after imputer.transform):")
print(pd.DataFrame(X_train_imputed_nom_demo, columns=nom_col_demo, index=X_train.index).iloc[40:45])
print("\nOne-Hot Encoded X_train 'job' shape:", X_train_onehot_encoded_demo.shape)
print("First 5 rows of One-Hot Encoded X_train 'job':")
print(pd.DataFrame(X_train_onehot_encoded_demo, columns=onehot_encoder.get_feature_names_out(nom_col_demo), index=X_train.index).iloc[40:45])

### **Exercise 1: Apply All Preprocessing & Train Logistic Regression**

Now, it's your turn to apply these preprocessing steps to *all* relevant columns and then train a Logistic Regression model.

**Instructions:**

1.  Look at the Variable Table in [this link](https://archive.ics.uci.edu/dataset/222/bank+marketing).
2. Make lists for `numerical_features`, `ordinal_features`, and `nominal_features`.
3. Preprocess the features. It is safer to make a copy of `X_train` using:
   ```
   X_train_copy = X_train.copy()
   X_test_copy = X_test.copy()
   ```
   and preprocess `X_train_copy` instead.

   **For nominal features, concat the one-hot encoded features using [`pd.concat(..., axis=1)`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) and drop the old nominal features from the dataframe.**
4. Train Logistic Regression on the preprocessed `X_train_copy` and `y_train`.
5. Evaluate the Model:
    *   Make predictions on the preprocessed `X_test_copy`.
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?


In [104]:
bank_url = 'https://raw.githubusercontent.com/donlap/ds352-labs/main/bank.csv'

df = pd.read_csv(bank_url, sep=';', na_values=['unknown'])
df = df.drop(["emp.var.rate", "cons.price.idx", "cons.conf.idx",	"euribor3m", "nr.employed"], axis=1)
print("Shape of the dataset:", df.shape)
df.head()

Shape of the dataset: (41188, 16)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,no


In [105]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y'], dtype='object')

In [106]:
df.dtypes

,0
age,int64
job,object
marital,object
education,object
default,object
housing,object
loan,object
contact,object
month,object
day_of_week,object


In [109]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [108]:
# --- YOUR CODE FOR EXERCISE 1 STARTS HERE ---
numerical_features = ['age', 'duration', 'campaign', 'pdays', 'previous']
ordinal_features = ['education']
nominal_features = ['job', 'marital', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']

In [110]:
X_train_copy = X_train.copy()
X_test_copy = X_test.copy()

In [111]:
scaler = StandardScaler()

X_train_copy[numerical_features] = scaler.fit_transform(X_train_copy[numerical_features])
X_test_copy[numerical_features] = scaler.transform(X_test_copy[numerical_features])

In [112]:
imputer_ord = SimpleImputer(strategy='most_frequent')

X_train_copy[ordinal_features] = imputer_ord.fit_transform(X_train_copy[ordinal_features])
X_test_copy[ordinal_features] = imputer_ord.transform(X_test_copy[ordinal_features])


In [114]:
education_categories = ['illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
                        'professional.course', 'university.degree', 'masters', 'doctorate']

In [115]:
ordinal_encoder = OrdinalEncoder(categories=[education_categories])

X_train_copy[ordinal_features] = ordinal_encoder.fit_transform(X_train_copy[ordinal_features])
X_test_copy[ordinal_features] = ordinal_encoder.transform(X_test_copy[ordinal_features])

In [116]:
X_train_copy['education']

,education
7564,5.0
23985,5.0
15127,3.0
1212,4.0
14186,4.0
...,...
24302,4.0
20900,1.0
30243,5.0
31520,1.0


In [117]:
X_train_dummies = pd.get_dummies(X_train_copy[nominal_features], drop_first=True)
X_test_dummies = pd.get_dummies(X_test_copy[nominal_features], drop_first=True)

In [118]:
X_train_copy = pd.concat([X_train_copy.drop(columns=nominal_features), X_train_dummies], axis=1)
X_test_copy = pd.concat([X_test_copy.drop(columns=nominal_features), X_test_dummies], axis=1)

In [119]:
print("--- Missing Values Count ---")
print(X_train_copy.isnull().sum())

--- Missing Values Count ---
age                     0
education               0
duration                0
campaign                0
pdays                   0
previous                0
job_blue-collar         0
job_entrepreneur        0
job_housemaid           0
job_management          0
job_retired             0
job_self-employed       0
job_services            0
job_student             0
job_technician          0
job_unemployed          0
marital_married         0
marital_single          0
default_yes             0
housing_yes             0
loan_yes                0
contact_telephone       0
month_aug               0
month_dec               0
month_jul               0
month_jun               0
month_mar               0
month_may               0
month_nov               0
month_oct               0
month_sep               0
day_of_week_mon         0
day_of_week_thu         0
day_of_week_tue         0
day_of_week_wed         0
poutcome_nonexistent    0
poutcome_success        0
dtype: in

In [128]:
X_train_copy = X_train_copy.drop(columns=['default_yes'])
X_train_copy

,age,education,duration,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_married,marital_single,housing_yes,loan_yes,contact_telephone,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success
7564,0.285300,5.0,-0.221060,-0.558189,0.194698,-0.348135,False,False,False,False,False,False,False,False,True,False,False,False,True,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False
23985,0.476579,5.0,-0.924021,2.623813,0.194698,-0.348135,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False
15127,-0.479817,3.0,-0.129370,0.148922,0.194698,-0.348135,True,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False
1212,-0.958015,4.0,-0.683334,-0.558189,0.194698,-0.348135,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,True,False
14186,-0.479817,4.0,0.084575,0.148922,0.194698,-0.348135,True,False,False,False,False,False,False,False,False,False,True,False,True,True,True,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24302,0.763498,4.0,-0.259265,-0.204633,0.194698,-0.348135,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,True,False
20900,0.859138,1.0,-0.205779,-0.558189,0.194698,-0.348135,True,False,False,False,False,False,False,False,False,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,True,False
30243,-0.384177,5.0,0.027268,-0.204633,-5.157614,3.717109,False,False,False,False,False,False,False,False,True,False,True,False,True,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True
31520,-0.862376,1.0,2.052101,-0.204633,0.194698,-0.348135,True,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,True,False


In [121]:
X_test_copy

,age,education,duration,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_married,marital_single,housing_yes,loan_yes,contact_telephone,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success
12239,-1.149295,4.0,-0.759742,-0.558189,0.194698,-0.348135,True,False,False,False,False,False,False,False,False,False,True,False,True,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False
7720,-0.575457,5.0,0.126600,0.502478,0.194698,-0.348135,False,True,False,False,False,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False
16560,-0.958015,2.0,3.412178,-0.204633,0.194698,-0.348135,True,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,True,True,False
6430,0.189660,4.0,1.368243,-0.558189,0.194698,-0.348135,False,False,False,False,False,False,False,False,True,False,False,True,True,False,True,False,False,False,False,False,True,False,False,False,False,False,False,True,True,False
38467,-0.671096,4.0,-0.755922,-0.204633,-5.136140,3.717109,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14660,-1.149295,2.0,0.015807,0.148922,0.194698,-0.348135,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,True,False
1617,1.050417,4.0,-0.427364,-0.558189,0.194698,-0.348135,True,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False
32438,-0.575457,6.0,-0.820869,-0.204633,0.194698,1.684487,True,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
20708,-0.001619,4.0,-0.534336,-0.558189,0.194698,-0.348135,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,True,True,False


In [129]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_copy, y_train)

LogisticRegression(max_iter=1000)

In [130]:
y = model.predict(X_test_copy)

In [131]:
print(classification_report(y_test, y))

              precision    recall  f1-score   support

           0       0.92      0.97      0.95     10976
           1       0.64      0.36      0.46      1381

    accuracy                           0.91     12357
   macro avg       0.78      0.67      0.71     12357
weighted avg       0.89      0.91      0.89     12357



## Part 2: Fashion-MNIST Dataset - Image Classification

### Load Fashion-MNIST Dataset

The Fashion-MNIST dataset consists of 28x28 grayscale images of fashion items.

In [ ]:
(fm_X_train, fm_y_train), (fm_X_test, fm_y_test) = fashion_mnist.load_data()

print(f"Fashion-MNIST Train data shape: {fm_X_train.shape}")
print(f"Fashion-MNIST Train labels shape: {fm_y_train.shape}")
print(f"Fashion-MNIST Test data shape: {fm_X_test.shape}")
print(f"Fashion-MNIST Test labels shape: {fm_y_test.shape}")

In [ ]:
print(f"First image {fm_X_train[0]}")
print(f"First label {fm_y_train[0]}")

### Visualize Fashion-MNIST Images

Let's see what these images look like.

In [ ]:
fashion_mnist_class_names = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

# Visualize the images
## Write your code here



### **Exercise 2: Preprocessing Images (Flatten and Scale)**

Images are 2D arrays (matrices of pixels) and pixel values are integers from 0-255. For Logistic Regression, we need:
*  **Flattening:** Convert each 28x28 image into a 1D array of 784 features.
*  **Scaling:** Normalize pixel values from [0, 255] to [0, 1].

**Instructions:**

1.   **Flatten:** Use the `.reshape()` method (see [documentation](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.reshape.html)). For `fm_X_train_binary` (shape `(num_samples, 28, 28)`), you want to reshape it to `(num_samples, 28*28)`.
2.  **Scale:** Divide the flattened pixel values by 255.0 to get values between 0 and 1.
3.   **Train Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
    *   Fit the model on your *processed* `fm_X_train_scaled` and `fm_y_train`.
4.   **Make Predictions:** Use `predict()` to make predictions on the *processed* `fm_X_test_scaled`.
5.   **Print Classification Report:** Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?
6.   **Visualize Misclassifications:**
    *   Find the indices in `fm_X_test_binary` where your model made incorrect predictions (i.e., `fm_y_pred != fm_y_test`).
    *   Select 5 of these misclassified images.
    *   Plot these images (using `plt.imshow`). For each image, print its true label and its predicted label.

In [ ]:
# --- YOUR CODE FOR EXERCISE 2 STARTS HERE ---





## Part 3: 20 Newsgroups Dataset - Text Classification

### Load 20 Newsgroups Dataset

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics.

In [ ]:
news_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
news_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

X_train_news, y_train_news = news_train.data, news_train.target
X_test_news, y_test_news = news_test.data, news_test.target

print(f"Number of training documents: {len(X_train_news)}")
print(f"Number of test documents: {len(X_test_news)}")
print(f"Categories: {news_train.target_names}")

### Explore Sample Document

In [ ]:
# Print the first document and its class
## Write your code here



### Preprocessing: Text Vectorization Demonstration with `TfidfVectorizer`

$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$

Where:

$$
\text{TF}(t, d) = \frac{\text{number of word }t\text{ in } d}{\text{number of words in } d} \quad \text{ and } \quad
\text{IDF}(t, D) = \log\left(\frac{\text{total number of documents}}{\text{number of documents that contain word }t}\right).
$$

In [ ]:
sample_sentences = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the sample sentences
sample_vec_output_sparse = # Write your code here

sample_vec_output_dense = sample_vec_output_sparse.toarray()

print(vectorizer.vocabulary_)
print(vectorizer.get_feature_names_out())
print(sample_vec_output_dense)

### **Exercise 3: Apply TF-IDF Vectorization to Full Dataset**

Now, apply `TfidfVectorizer` to the actual training and testing datasets for the 20 Newsgroups classification task.

**Instructions:**

1.  **Initialize `TfidfVectorizer`:**
    *   Initialize `TfidfVectorizer`. Use `stop_words='english'` to remove common words.
2.  **Fit and Transform Training Data:**
    *   Call `fit_transform()` on `X_train_news` to learn the vocabulary and transform the training text into TF-IDF features. Store the result in `X_train_vec`.
3.  **Transform Test Data:**
    *   Call `transform()` on `X_test_news` using the *already fitted* vectorizer. Store the result in `X_test_vec`. **Crucially, do not call `fit_transform()` on the test data!** This would cause data leakage.
4.  **Initialize Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
5.  **Train the Model:**
    *   Fit the model on your `X_train_vec` and `y_train_news`.
6.  **Make Predictions:**
    *   Make predictions using `predict()` on the `X_test_vec`.
7.  **Evaluate the Model:**
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?

In [ ]:
# --- YOUR CODE FOR EXERCISE 3 STARTS HERE ---


